In [1]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/final.csv')
df = df[['title','body', 'label']]
df.head()

,title,body,label
0,Any Advice On How to Not Be A Bitter PhD Student?,I'm not a PhD student but I am on my way to ob...,0
1,What is the true state of humanities academia?,Historian here. I know quite a few historians ...,0
2,A friend has offered to give me some Ostrich a...,Ostrich Eggs are awesome and easy. I make at l...,0
3,When did humans begin using blankets/sheets an...,Humans have been using blankets and sleeping i...,1
4,Anywhere to find reference for pig bones?,It exists! This looks good: https://peterborou...,0


In [2]:
# Replace symbols in the 'body' column
df['body'] = df['body'].str.replace('â€™', "'")
df['title'] = df['title'].str.replace('â€™', "'")
# Display the first 20 rows
df.head(20)

,title,body,label
0,Any Advice On How to Not Be A Bitter PhD Student?,I'm not a PhD student but I am on my way to ob...,0
1,What is the true state of humanities academia?,Historian here. I know quite a few historians ...,0
2,A friend has offered to give me some Ostrich a...,Ostrich Eggs are awesome and easy. I make at l...,0
3,When did humans begin using blankets/sheets an...,Humans have been using blankets and sleeping i...,1
4,Anywhere to find reference for pig bones?,It exists! This looks good: https://peterborou...,0
5,Is Hierarchy and Power (in a social context) h...,"In my opinion, the concept of hierarchy and po...",1
6,I feel unloved constantly. Venus squares. What...,"i'm not an expert, but my input:\n\nvenus in a...",0
7,Despite breaking up and getting back together ...,"You are in an on-again, off-again relationship...",0
8,What are some realistic career options in fiel...,There are several realistic career options in ...,1
9,Should I join my supervisor starting a new res...,"Well, that's a tough decision to make! Joining...",1


In [3]:
df['label'].value_counts()

0    6948
1    6948
Name: label, dtype: int64

In [4]:
# Create a new column 'input' by concatenating 'title' and 'body' with a colon separator
combined_text = df['title'] + ' ' + df['body']
labels = df['label']

In [5]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.2 MB/s eta 0:00:00


In [6]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

In [7]:
# Step 2: Split the Data
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(combined_text, labels, test_size=0.2, random_state=42)

In [8]:
# Step 3: Tokenization and Formatting
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, return_tensors='pt', max_length=512)
val_encodings = tokenizer(list(X_val), truncation=True, padding=True, return_tensors='pt', max_length=512)

In [9]:
train_dataset = TensorDataset(train_encodings.input_ids, train_encodings.attention_mask, torch.tensor(y_train.values))
val_dataset = TensorDataset(val_encodings.input_ids, val_encodings.attention_mask, torch.tensor(y_val.values))


In [10]:
# Step 4: Model Training
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [11]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [12]:
# Step 5: Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Step 6: Training loop
best_val_loss = float('inf')
num_epochs = 5  # Adjust this as needed

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_train_loss / len(train_loader)

    # Step 7: Validation loop
    model.eval()
    total_val_loss = 0

    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(test_loader)

    print(f'Epoch {epoch + 1}:')
    print(f'Training Loss: {avg_train_loss:.4f}')
    print(f'Validation Loss: {avg_val_loss:.4f}')

    # Step 8: Save the best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), 'best_model_without_dropout.pth')

print('Training complete.')

Epoch 1:
Training Loss: 0.0497
Validation Loss: 0.0303
Epoch 2:
Training Loss: 0.0046
Validation Loss: 0.0056
Epoch 3:
Training Loss: 0.0042
Validation Loss: 0.0073
Epoch 4:
Training Loss: 0.0002
Validation Loss: 0.0064
Epoch 5:
Training Loss: 0.0001
Validation Loss: 0.0088
Training complete.


In [13]:
import numpy as np
# Load the best model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.load_state_dict(torch.load('best_model_without_dropout.pth'))
model.to(device)
model.eval()

# Define a DataLoader for your test data (similar to how you did for validation)
# test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Initialize lists to store predictions and true labels
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        predictions.extend(torch.argmax(logits, dim=1).tolist())
        true_labels.extend(labels.tolist())

# Convert the predictions and true labels to numpy arrays for further analysis
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Calculate accuracy
accuracy = np.mean(predictions == true_labels)
print(f'Accuracy: {accuracy * 100:.2f}%')

# You can also use classification metrics like precision, recall, F1-score, etc. depending on your specific use case.

# Optional: Print a classification report
from sklearn.metrics import classification_report
target_names = ['Class 0', 'Class 1']  # Replace with your actual class names
print(classification_report(true_labels, predictions, target_names=target_names))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy: 99.86%
              precision    recall  f1-score   support

     Class 0       1.00      1.00      1.00      1440
     Class 1       1.00      1.00      1.00      1340

    accuracy                           1.00      2780
   macro avg       1.00      1.00      1.00      2780
weighted avg       1.00      1.00      1.00      2780



In [14]:
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        predictions.extend(torch.argmax(logits, dim=1).tolist())
        true_labels.extend(labels.tolist())

# Convert the predictions and true labels to numpy arrays for further analysis
predictions = np.array(predictions)
true_labels = np.array(true_labels)


# Display some test cases and their corresponding outputs
num_cases_to_display = 10
for i in range(num_cases_to_display):
    print(f'Test Case {i+1}:')
    print(f'Input Text: {X_val.iloc[i]}')  # Corrected access to input text
    print(f'Predicted Class: {predictions[i]}')
    print(f'True Class: {true_labels[i]}')
    print('\n')

Test Case 1:
Input Text: How does scholarly consensus work in the humanities and social sciences? Scholarly consensus in the humanities and social sciences is a fascinating and complex process. Unlike in the natural sciences, where experiments and empirical evidence often lead to more definitive conclusions, consensus in these fields is often more nuanced and subject to ongoing debate.

In the humanities, such as literature, philosophy, and history, consensus is typically reached through a combination of rigorous analysis, interpretation, and critical discourse. Scholars engage in extensive research, examining primary and secondary sources, and presenting their findings through publications and academic conferences. Through this process, they contribute to the existing body of knowledge and engage in dialogue with their peers.

In the social sciences, which include disciplines like sociology, psychology, and anthropology, consensus is often built through empirical research, statistical

In [15]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [17]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.load_state_dict(torch.load('best_model_without_dropout.pth'))
model.to(device)
model.eval()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
inputs = [
    "I was just reading up on the ancient Theia planet that supposedly collided with earth, it likely had water, would it have had life? The short version is (1) we don't know and there's effectively no way we could know, but (2) it's broadly unlikely. For the first part, it's important to consider just how violent and destructive the impact between Theia and the Proto-Earth was. This is discussed in detail in a variety of publications, but the recent paper by Yuan et al., 2023 provides a nice summary and graphic (their Figure 1). Specifically, the impact between these two planets effectively completely melted the crust and much of the mantle of both Proto-Earth and Theia, with the core of Theia (and portions of Theia's mantle based on the results of Yuan et al) sinking and mixing with the Proto-Earth core / lower mantle. If there was hypothetically life on either Proto-Earth, Theia, or both, suffice to say, it would been eradicated during this event and all evidence would have been destroyed during the extreme melting and segregation processes that formed Earth as we know it (in terms of mass, etc.) and the Moon. For the second part, it's useful to consider the timeframes in involved. The impact of Theia with the Proto-Earth and the subsequent formation of the Moon, happened very early in the history of the solar system. The exact timing has been updated a few times, but recent results from Greer et al., 2023 suggest that this happened only ~110 million years after the formation of the solar system, or about 4.46 billion years ago. If we consider evidence for formation of life on Earth, whether we're thinking of the oldest preserved microfossil s(e.g., Schopf et al., 2017) or preservation of biosignatures more broadly (e.g., Homann et al., 2019), the earliest dates are ~3.5 billion years ago, i.e., nearly a full billion years after the Moon forming impact. It's hard to extrapolate from a dataset of 1, but if we consider that it took ~1 billion years for life to develop on Earth and that Theia as a planet had only existed for ~100 million years before it impacted the Proto-Earth, it becomes relatively unlikely that sufficient time had past for life to develop on either body prior to their collision. Even less so if we consider that this early period of the solar system would have been very chaotic, with lots of impacts from planetisemals and the like disrupting the surfaces of most every planetary body frequently.",
    "I was just reading up on the ancient Theia planet that supposedly collided with earth, it likely had water, would it have had life? The ancient Theia planet is a fascinating subject in the realm of planetary science. According to the Giant Impact Hypothesis, it is believed that Theia, a Mars-sized protoplanet, collided with early Earth around 4.5 billion years ago, leading to the formation of the Moon. This cataclysmic event had profound implications for the evolution of our planet. One intriguing aspect of Theia is the possibility that it harbored water. The presence of water on a celestial body is a crucial factor in the potential for life to exist. Water serves as a universal solvent, providing a medium for chemical reactions necessary for life as we know it. If Theia did indeed possess water, it would have increased the likelihood of life, at least in microbial form, having emerged on the planet. However, the question of whether Theia supported life is a complex one. While water is a crucial ingredient for life, it is not the sole determining factor. Other conditions, such as temperature, chemical composition, and the availability of organic molecules, also play pivotal roles. Theia's environment prior to the collision with Earth would have influenced its potential for life. The conditions on Theia prior to the impact are subject to speculation. It likely existed in the early solar system, where conditions were turbulent and dynamic. The protoplanetary disk from which Theia formed contained a mix of materials, including volatile compounds like water. This could suggest that Theia had the potential for habitability, at least in theory. However, it's important to note that the collision with Earth would have been a cataclysmic event. The immense energy released in the impact would have sterilized both Theia and Earth, obliterating any existing life forms. This renders the question of life on Theia moot in the context of the collision itself. In conclusion, while the presence of water on Theia may have increased the potential for life, the collision with Earth would have erased any such possibilities. The ancient Theia planet remains a captivating subject of scientific inquiry, offering insights into the early history of our solar system. The hypothetical existence of life on Theia, prior to the collision, highlights the intricate interplay of factors that determine habitability on celestial bodies."
    # Add more sentences if needed
    ]

In [19]:
encodings = tokenizer(inputs, truncation=True, padding=True, return_tensors='pt', max_length=512)
input_ids = encodings.input_ids.to(device)
attention_mask = encodings.attention_mask.to(device)

In [20]:
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).tolist()

In [21]:
for i, prediction in enumerate(predictions):
    print(f'Sentence {i+1} - Predicted Class: {prediction}')

Sentence 1 - Predicted Class: 0
Sentence 2 - Predicted Class: 1
